## Translate

In [1]:
from google_trans_new import google_translator  
import pandas as pd
import json
import os
import re
import unicodedata
import time

In [3]:
def clean_text(text):
    text = unicodedata.normalize("NFKD", text)
    text = re.sub("\xad", "", text)
    
    # hapus titik dibeberapa kata tertentu
    text = re.sub(" Tbk\.", " Tbk ", text)
    text = re.sub(" Tbk", " Tbk ", text)
    text = re.sub(" Rp\.", " ", text)
    text = re.sub(" Rp", " ", text)
    text = re.sub(" PT\.", " PT ", text)
    text = re.sub(" Pt\.", " PT ", text)
    text = re.sub(" dr\.", " ", text)
    text = re.sub(" Dr\.", " ", text)
    text = re.sub(" DR\.", " ", text)
    text = re.sub(" N\.A\.", " ", text)
    text = re.sub(" H\.M\.", " HM", text)    
    text = re.sub(" jl\. ", " jalan ", text)
    text = re.sub(" Jl\. ", " jalan ", text)
    text = re.sub(" Jln\. ", " jalan ", text)
    text = re.sub(" jln\. ", " jalan ", text)    

    # stop words
    text = re.sub("Berikut rincian kurs jual-beli.*$", "", text)
    text = re.sub("(Simak berita lainnya seputar topik.+)$", "", text)
    text = re.sub("--.*--", " ", text) # hapus semua kalimat yang ada di tengah -- dan --
    
    # others pattern
    text = re.sub(r"\. *[0-9]+\. ", ". ", text) # hapus angka yang menunjukkan list seperti 1. 2. yang diawali dengan titik
    text = re.sub(r": *[0-9]+\. ", ". ", text) # hapus angka yang menunjukkan list seperti 1. 2. yang diawali dengan ":"
    text = re.sub("(?<=\d)(?=[a-zA-Z])", " ", text) # memisahkan angka yang berdempetan dengan huruf
    text = re.sub("(?<=[a-zA-Z])(?=\d)", " ", text) # memisahkan huruf yang berdempetan dengan angka
    text = re.sub(" ([a-zA-Z]{1,2})\. ", " ", text) # hapus dua huruf yang diikuti titik karena biasa hanya singkatan nama
    
    # hapus url
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "link-website", text)
    
    text = re.sub("\*", "", text) # hapus bintang
    text = re.sub(r"(?<=\d)[,.](?=\d)", "", text) # hapus titik atau koma ditengah angka
    text = re.sub(":", ": ", text) # menambahkan spasi setelah :
    text = re.sub(r"\.(?=\S)", ". ", text) # ada spasi setelah titik
    text = re.sub(r"\?(?=\S)", "? ", text) # ada spasi setelah tanda tanya
    text = re.sub(r"\)(?=\S)", ") ", text) # ada spasi setelah tutup kurung
    text = re.sub(r"\((?=\S)", " (", text) # ada spasi sebelum tutup kurung

    text = re.sub("\. +\.", ".", text) # hapus spasi diantara dua titik
    text = re.sub(" +\? +", "? ", text) # hapus spasi sebelum tanda tanya
    text = re.sub(" +,", ", ", text) # hapus spasi sebelum koma
    text = re.sub(" +\. ", ". ", text) # hapus spasi sebelum titik
    text = re.sub("\.,", ", ", text) # hapus .,
    
    text = re.sub(" {2,}", " ", text) # hapus spasi yang berulang
    text = re.sub("\.{2,}", ".", text) # hapus titik yang berulang
    text = re.sub("\?{2,}", ".", text) # hapus tanda tanya yang berulang
    
    text = re.sub(r'\. *\([^)]*\)[\. ]', ". ", text) # hapus kurung di awal kalimat
    text = re.sub(r'\. *\([^)]*\)$', ".", text) # hapus kurung di akhir artikel
    
    text = re.sub(" Bisnis\.com", " bisniscom", text)
    text = re.sub(" Bisnis\. com", " bisniscom", text)
    text = re.sub(" bisnis\. com", " bisniscom", text)
    text = re.sub(" bisnis\.com", " bisniscom", text)
    
    return text.strip()

data = pd.read_json("data/table_articles.json")
data['content_clean'] = data.content.apply(clean_text)

In [2]:
class Translator():
    """
    Class ini digunakan untuk melakukan translate menggunakan 
    package `google_translator()`. Untuk melakukan translate
    gunakan method `translate`
    """
    
    def __init__(self, fail_max=5):
        """
        Argumen `fail_max` digunakan untuk menentukan jumlah
        maksimal error sehingga proses translate akan dihentikan
        """
        self.__init_translator()
        self.fail_max = fail_max
        
    def __init_translator(self):
        self.translator = google_translator()
        
    def __translate_text(self, text, src='id', tgt="en"):
        """
        Method untuk mentranslate satu teks, jika terjadi error
        maka akan dilakukan sleep selama 60 detik dan dilakukan
        inisialisasi lagi `google_translator`
        """
        fail_count = 0
        while True:
            try:
                return self.translator.translate(text, lang_src=src, lang_tgt=tgt)
            except Exception as e:
                fail_count += 1
                print(str(e))
                
                if fail_count == self.fail_max:
                    print("Maximum fail has been exceeded")
                    raise e
                    
                time.sleep(60)
                self.__init_translator()
    
    def __split_text(self, text):
        """
        Private method ini digunakan untuk memecah teks yang
        panjangnya lebih dari 5000 karakter menjadi beberapa bagian.
        Hal ini dilakukan karena google translate hanya bisa
        melakukan translate untuk 5000 karakter. 
        
        Memecah teksnya juga berdasarkan kalimat yang dipisahkan oleh titik.
        Misalnya pada karakter 5000 bukan titik, maka akan dicari titik 
        sebelum karakter 5000 dan teksnya akan dipecah mulai titik tersebut.
        """
        start = 0
        end = 0
        len_text = len(text)
        
        translated_data = []
        while end != len_text:
            end = start + 4999
            if end > len(text):
                end = len(text)
            else:
                # Cari posisi titik terakhir sebelum karakter ke 5000
                while text[end-1] != ".":
                    end -= 1
                    
                # Jika dalam 5000 kata tersebut tidak ada titik maka
                # kembalikan None yang artinya teks gagal untuk ditranslate
                if end == start:
                    return None

            translated_data.append(text[start:end].strip())
            start = end
        return translated_data

    def translate(self, text, src='id', tgt="en"):
        """
        Method utama dari Class ini yang merupakan interface
        untuk melakukan translate suatu teks. `src` merupakan 
        bahasa dari teks yang akan di translate dan `tgt` merupakan
        target bahasanya.
        """
        
        # Jika panjang teksnya lebih dari 5000 maka split
        # teksnya menjadi beberapa bagian
        if len(text) >= 5000:
            translated_data = self.__split_text(text)
            if translated_data is not None:
                return " ".join([self.__translate_text(sentance, src, tgt) for sentance in translated_data])
            return None
        else:
            return self.__translate_text(text, src, tgt)

Jika terdapat error [Python google-trans-new translate raises error: JSONDecodeError: Extra data:](https://stackoverflow.com/questions/68214591python-google-trans-new-translate-raises-error-jsondecodeerror-extra-data)

In [ ]:
try:
    with open("data/translate.txt", "r", encoding='utf-8') as f:
        data_translate = json.load(f)
except:
    data_translate = []
    
link = [article['link'] for article in data_translate]

translator = Translator()
iterasi = len(data_translate)

for index, row in data[~data.link.isin(link)].iterrows():
    print(f"{iterasi}. {row['link']}")
    
    try:
        translate_text = translator.translate(row['content_clean'])
    except Exception as e:
        with open("data/translate.txt", "w", encoding='utf-8') as f:
            json.dump(data_translate, f, ensure_ascii=False)
        break

    data_translate.append({'link' : row['link'], 'translate_text' : translate_text})
    iterasi += 1
    if iterasi % 50 == 0:
        with open("data/translate.txt", "w", encoding='utf-8') as f:
            json.dump(data_translate, f, ensure_ascii=False)
        time.sleep(10)

In [6]:
with open("data/translate.txt", "w", encoding='utf-8') as f:
    json.dump(data_translate, f, ensure_ascii=False)